In [1]:
import ROOT
from os import path
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import numpy as np
from statistics import mean, median
import math
from array import array
import csv
from datetime import datetime
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as bck

def r2(ytrue, ypred):
    print(type(ytrue))
    print(type(ypred))
    res = bck.sum(bck.square(ytrue-ypred))
    tot = bck.sum(bck.square(ytrue-bck.mean(ytrue)))
    return 1-res/tot

def Build_ML_Feature_Arrays_ptTrue(
    input_file_path, input_tree_name, pt_true_min, pt_true_max) :
    """
    Creates an array of all features exported from ROOT.
    Applies a cut using pT_True values between pt_true_min and pt_true_max.
    """

    input_file = None;
    if (ROOT.gSystem.AccessPathName(input_file_path)) :
        print("Input file path does not exist:", input_file)
        exit()
    else :
        input_file = ROOT.TFile.Open(input_file_path, "READ")
        print("Input file accessed successfully. Output file generated.")
    
    print("Accessing input tree...")
    input_tree = input_file.Get(input_tree_name)
    print("Input tree accessed successfully.")
    
    # Setup Arrays
    X_values_A  = []  # Array of arrays of inputs corresponding to pT_true as PYTHIA jet pT
    y_values_A  = []  # Array of targets for regression, pT_true is PYTHIA jet pT
    
    X_values_B  = []  # Array of arrays of inputs corresponding to pT_true as jet pT * PYTHIA pT / const. pT
    y_values_B  = []  # Array of targets for regression, pT_true is jet pT * PYTHIA pT / const. pT

    # Predictors
    jet_pt_raw       = None  # Raw/uncorrected jet pt
    jet_pt_corr      = None  # Corrected jet pt
    jet_mass         = None
    jet_area         = None
    jet_area_err     = None
    jet_const_n      = None
    const_pt_mean    = None  # Mean pt of jet constituents
    const_pt_median  = None  # Mean pt of jet constituents
    const_1_pt       = None  # pt of jet constituent particle 1
    const_2_pt       = None  # pt of jet constituent particle 2
    const_3_pt       = None  # pt of jet constituent particle 3
    const_4_pt       = None  # pt of jet constituent particle 4
    const_5_pt       = None  # pt of jet constituent particle 5
    const_6_pt       = None  # pt of jet constituent particle 6
    const_7_pt       = None  # pt of jet constituent particle 7
    const_8_pt       = None  # pt of jet constituent particle 8
    const_9_pt       = None  # pt of jet constituent particle 9
    const_10_pt      = None  # pt of jet constituent particle 10
    jet_y            = None
    jet_phi          = None
    jet_rho          = None

    # Targets
    jet_pt_true_A    = None  # True jet pt (determined from PYTHIA jets)
    jet_pt_true_B    = None

    # Helper Variables
    event_counter    = 0
    event_n_total    = 20000
    jet_n            = None  # Number of jets in an event
    jet_n_counter_A  = 0
    jet_n_counter_B  = 0
    jet_const_pt_arr = []    # Array of jet constituents and their values
    sc_correction_arr_A = []
    sc_correction_arr_B = []
    
    print("Preparing to collect data from tree...")

    # Collecting from TTree
    for event in input_tree :  
        jet_n = event.jet_n

        for jet in range(0, 2) :
            jet_pt_raw      = input_tree.jet_pt_raw[jet]
            jet_pt_corr     = input_tree.jet_pt_corr[jet]
            jet_mass        = input_tree.jet_mass[jet]
            jet_area        = input_tree.jet_area[jet]
            jet_const_n     = input_tree.jet_const_n[jet]
            const_pt_mean   = input_tree.const_pt_mean[jet]
            const_pt_median = input_tree.const_pt_median[jet]
            const_1_pt      = input_tree.const_1_pt[jet]
            const_2_pt      = input_tree.const_2_pt[jet]
            const_3_pt      = input_tree.const_3_pt[jet]
            const_4_pt      = input_tree.const_4_pt[jet]
            const_5_pt      = input_tree.const_5_pt[jet]
            const_6_pt      = input_tree.const_6_pt[jet]
            const_7_pt      = input_tree.const_7_pt[jet]
            const_8_pt      = input_tree.const_8_pt[jet]
            const_9_pt      = input_tree.const_9_pt[jet]
            const_10_pt     = input_tree.const_10_pt[jet]
            jet_y           = input_tree.jet_y[jet]
            jet_phi         = input_tree.jet_phi[jet]
            jet_rho         = input_tree.jet_rho[jet]
            
            jet_pt_true_A   = input_tree.jet_pt_true_pythia[jet]
            jet_pt_true_B   = input_tree.jet_pt_true_paper[jet]
            
            temp_jet_arr = [
                    jet_pt_corr,      jet_pt_raw,       jet_area, jet_mass, 
                        jet_const_n,     const_pt_mean,   const_pt_median, jet_rho,
                    const_1_pt,      const_2_pt,      const_3_pt,      const_4_pt,
                    const_5_pt,      const_6_pt,      const_7_pt,      const_8_pt,
                    const_9_pt,      const_10_pt,     jet_y,           jet_phi]
            
            if (jet_pt_true_A != 0.0) and (jet_pt_true_A > pt_true_min) and (jet_pt_true_A < pt_true_max) :
                X_values_A.append(temp_jet_arr)

                y_values_A.append(jet_pt_true_A)
                
                sc_correction_arr_A.append(jet_pt_corr)
                
                jet_n_counter_A   = jet_n_counter_A + 1
                
                if event_counter % 1000 == 0 : print(f"Event: {event_counter:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_A: {jet_pt_true_A: 5.3f}")
            
            if (jet_pt_true_B != 0.0) and (jet_pt_true_B > pt_true_min) and (jet_pt_true_B < pt_true_max) :
                X_values_B.append(temp_jet_arr)
                
                y_values_B.append(jet_pt_true_B)

                sc_correction_arr_B.append(jet_pt_corr)
                
                jet_n_counter_B   = jet_n_counter_B + 1

                if event_counter % 1000 == 0 : print(f"Event: {event_counter:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_B: {jet_pt_true_B: 5.3f}")
            
        event_counter += 1

    print(f"All data transferred to array. Testing with {jet_n_counter_A} A-jets and {jet_n_counter_B} B-jets.\n")
    print(f"Training set A: {len(X_values_A)} / {len(y_values_A)} / {len(sc_correction_arr_A)}")
    print(f"Training set B: {len(X_values_B)} / {len(y_values_B)} / {len(sc_correction_arr_B)}")

    input_file.Close()
    print("Input file closed.")
    
    return X_values_A, y_values_A, sc_correction_arr_A, X_values_B, y_values_B, sc_correction_arr_B

    

def Build_ML_Feature_Arrays_ptCorr(
    input_file_path, input_tree_name, pt_corr_min, pt_corr_max) :
    """
    WARNING: CODE MAY BE OUTDATED
    Creates an array of all features exported from ROOT.
    Applies a cut using pT_Corrected values between pt_corr_min and pt_corr_max.
    This was used once for an alternative selection method and may no longer work.
    """

    input_file = None;
    if (ROOT.gSystem.AccessPathName(input_file_path)) :
        print("Input file path does not exist:", input_file)
        exit()
    else :
        input_file = ROOT.TFile.Open(input_file_path, "READ")
        print("Input file accessed successfully. Output file generated.")
    
    print("Accessing input tree...")
    input_tree = input_file.Get(input_tree_name)
    print("Input tree accessed successfully.")

    # Setup Arrays
    X_values_A  = []  # Array of arrays of inputs corresponding to pT_true as PYTHIA jet pT
    y_values_A  = []  # Array of targets for regression, pT_true is PYTHIA jet pT
    
    X_values_B  = []  # Array of arrays of inputs corresponding to pT_true as jet pT * PYTHIA pT / const. pT
    y_values_B  = []  # Array of targets for regression, pT_true is jet pT * PYTHIA pT / const. pT

    # Predictors
    jet_pt_raw       = None  # Raw/uncorrected jet pt
    jet_pt_corr      = None  # Corrected jet pt
    jet_mass         = None
    jet_area         = None
    jet_area_err     = None
    jet_const_n      = None
    const_pt_mean    = None  # Mean pt of jet constituents
    const_pt_median  = None  # Mean pt of jet constituents
    const_1_pt       = None  # pt of jet constituent particle 1
    const_2_pt       = None  # pt of jet constituent particle 2
    const_3_pt       = None  # pt of jet constituent particle 3
    const_4_pt       = None  # pt of jet constituent particle 4
    const_5_pt       = None  # pt of jet constituent particle 5
    const_6_pt       = None  # pt of jet constituent particle 6
    const_7_pt       = None  # pt of jet constituent particle 7
    const_8_pt       = None  # pt of jet constituent particle 8
    const_9_pt       = None  # pt of jet constituent particle 9
    const_10_pt      = None  # pt of jet constituent particle 10
    jet_y            = None
    jet_phi          = None
    jet_rho          = None
    

    # Targets
    jet_pt_true_A    = None  # True jet pt (determined from PYTHIA jets)
    jet_pt_true_B    = None

    # Helper Variables
    event_n          = 0
    event_n_total    = 20000
    jet_n            = None  # Number of jets in an event
    jet_n_counter_A  = 0
    jet_n_counter_B  = 0
    jet_const_pt_arr = []    # Array of jet constituents and their values
    sc_correction_arr_A = []
    sc_correction_arr_B = []

    print("Preparing to collect data from tree...")
    
    # Collecting from TTree
    for event in input_tree :  
        jet_n = event.jet_n
        
        for jet in range(0, 2) :
            jet_pt_raw      = input_tree.jet_pt_raw[jet]
            jet_pt_corr     = input_tree.jet_pt_corr[jet]
            jet_mass        = input_tree.jet_mass[jet]
            jet_area        = input_tree.jet_area[jet]
            jet_const_n     = input_tree.jet_const_n[jet]
            const_pt_mean   = input_tree.const_pt_mean[jet]
            const_pt_median = input_tree.const_pt_median[jet]
            const_1_pt      = input_tree.const_1_pt[jet]
            const_2_pt      = input_tree.const_2_pt[jet]
            const_3_pt      = input_tree.const_3_pt[jet]
            const_4_pt      = input_tree.const_4_pt[jet]
            const_5_pt      = input_tree.const_5_pt[jet]
            const_6_pt      = input_tree.const_6_pt[jet]
            const_7_pt      = input_tree.const_7_pt[jet]
            const_8_pt      = input_tree.const_8_pt[jet]
            const_9_pt      = input_tree.const_9_pt[jet]
            const_10_pt     = input_tree.const_10_pt[jet]
            jet_y           = input_tree.jet_y[jet]
            jet_phi         = input_tree.jet_phi[jet]
            jet_rho         = input_tree.jet_rho[jet]
            
            jet_pt_true_A   = input_tree.jet_pt_true_pythia[jet]
            jet_pt_true_B   = input_tree.jet_pt_true_paper[jet]
            
            temp_jet_arr = [
                    jet_pt_raw,      jet_pt_corr,     const_pt_mean,        jet_area, 
                        jet_const_n,     jet_mass,   const_pt_median, jet_rho,
                    const_1_pt,      const_2_pt,      const_3_pt,      const_4_pt,
                    const_5_pt,      const_6_pt,      const_7_pt,      const_8_pt,
                    const_9_pt,      const_10_pt,     jet_y,           jet_phi,]
            
            if (jet_pt_true_A != 0.0) and (jet_pt_corr > pt_corr_min) and (jet_pt_corr < pt_corr_max) :
                
                X_values_A.append(temp_jet_arr)

                y_values_A.append(jet_pt_true_A)
                
                sc_correction_arr_A.append(jet_pt_corr)
                
                jet_n_counter_A   = jet_n_counter_A + 1
                
                if jet_n_counter_A % 10 == 0 :
                    print(f"Event: {event_n:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_A: {jet_pt_true_A: 5.3f}")
            
            if (jet_pt_true_B != 0.0) and (jet_pt_corr > pt_corr_min) and (jet_pt_corr < pt_corr_max) :
                X_values_B.append(temp_jet_arr)
                
                y_values_B.append(jet_pt_true_B)

                sc_correction_arr_B.append(jet_pt_corr)
                
                jet_n_counter_B   = jet_n_counter_B + 1

                if jet_n_counter_B % 10 == 0 :
                    print(f"Event: {event_n:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_B: {jet_pt_true_A: 5.3f}")

        event_n = event_n + 1

    print(f"All data transferred to array. Testing with {jet_n_counter_A} A-jets and {jet_n_counter_B} B-jets.\n")
    print(f"Training set A: {len(X_values_A)} / {len(y_values_A)} / {len(sc_correction_arr_A)}")
    print(f"Training set B: {len(X_values_B)} / {len(y_values_B)} / {len(sc_correction_arr_B)}")

    input_file.Close()
    print("Input file closed.")
    
    return X_values_A, y_values_A, sc_correction_arr_A, X_values_B, y_values_B, sc_correction_arr_B


def buildfeaturescsv(filename):
    x = []
    y = []
    with open(filename) as file:
        for line in file:
            temp = []
            line = line.strip().split(',')
            for i in range(1,len(line)):
                temp.append(float(line[i]))
            y.append(float(line[0]))
            x.append(temp)
    return x, y

Welcome to JupyROOT 6.26/06


2023-01-19 09:19:42.445395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 09:19:42.600139: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-19 09:19:42.603722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/qq/Documents/root_install/lib:/home/qq/fastjet-install/lib/
2023-01-19 09:19:42.603731: I 

In [2]:
X_train, y_train = [], []
powers = []
train = []
test = []
trainstring = ""
teststring = ""
with open("powers.txt") as file:
    for line in file:
        powers.append(int(line.strip()))

with open("ranges.txt") as file:
    line = file.readline()
    line = line.strip().split()
    for bound in line:
        train.append(int(bound))
    line = file.readline()
    line = line.strip().split()
    for bound in line:
        test.append(int(bound))

trainstring = str(train[0]) + "_" + str(train[1])
teststring = str(test[0]) + "_" + str(test[1])
            
train_tree_name    = "Tree_Tree"
train_file_paths = []
for power in powers:
    train_directory    = "./prepped/"
    train_file_name    = "ML_Prep_" + trainstring + "_Train" + str(power) +".csv"
    train_file_path    = train_directory + train_file_name
    train_file_paths.append(train_file_path)

for i in range(len(powers)):
    x, y = buildfeaturescsv(train_file_paths[i])
    X_train.append(x)
    y_train.append(y)
now = datetime.now()
dt_string = now.strftime("%Y/%m/%d %H:%M:%S")

print("Ready!", dt_string)

Ready! 2023/01/19 09:19:46


In [3]:
output_dir = "./compiled/Data"
output_fil = "results_" + trainstring
output_ext = ".root"

In [4]:
#counts,bins = np.histogram(newY, bins=9)
#plt.hist(bins[:-1], bins, weights=counts, range=(10,90))

def test(X_test, y_test, indices, model, i, l, k, q):
    y_pred = model.predict(X_test)
    xmin = -49.5
    xmax = 50.5
    bins = 100
    outf = ROOT.TFile.Open(output_directory + output_fil + "train_ptbias" + str(l) + "test_ptbias" + str(8) + model.__class__.__name__ + str(len(indices)) + "Features_set" + str(ptrange[k][0])+ "_" + str(ptrange[k][1])+ output_ext, "RECREATE")
    outt = ROOT.TTree(test_tree_names[0] + model.__class__.__name__, "tree")
    csvfile = output_directory + output_fil + "_train_ptbias" + str(l) + "_test_ptbias" + str(8) + "_" + model.__class__.__name__ + "_" + str(len(indices)) + "Features_set" + str(ptrange[k][0])+ "_" + str(ptrange[k][1]) + ".csv"
    f = open(csvfile, "w")
    deltahist = ROOT.TH1D("hist", "Jet p_{T} Delta " + str(len(indices)) + " Feature(s) Test " + test_tree_names[0][5:-5] + " GeV (Falling) Test 10_90 GeV (Falling) Train", bins, xmin, xmax)
    deltahist.GetXaxis().SetTitle("p_{T true} - p_{T predicted}")
    deltahist.GetYaxis().SetTitle("Counts")
    deltahist.GetXaxis().SetRangeUser(-50, 50)
    deltahist.GetYaxis().SetRangeUser(0,750)
    deltahist.SetMarkerStyle(39)
    deltahist.SetMarkerColor(2)
    for j in range(len(y_test)):
        if y_test[j] < ptrange[k][0] or y_test[j] > ptrange[k][1]:
            continue
        deltahist.Fill((y_pred[j] - y_test[j]))
        f.write(str((y_pred[j] - y_test[j])) + "\n")
    deltahist.SetDirectory(0)
    outf.cd()
    deltahist.Write()
    outf.Write()
    outf.Close()
    f.close()
    return 0

In [5]:
#y_train.append(newY)
#X_train.append(newX)

def fill_schist(corvals, truevals, indices, i, l, k):
    xmin = -49.5
    xmax = 50.5
    bins = 100 #NEED TO FIX: test ptbias needs to be selected from inputs, not hardcoded
    outf = ROOT.TFile.Open(output_directory + output_fil + "train_ptbias" + str(l) + "test_ptbias" + str(8) +"corrected" + str(len(indices)) + "Features_set" + str(ptrange[k][0])+ "_" + str(ptrange[k][1]) + output_ext, "RECREATE")
    outt = ROOT.TTree(test_tree_names[0][:-5] + " corrected", "tree")
    deltahist = ROOT.TH1D("hist", "Jet p_{T} Delta " + str(len(indices)) + " Feature(s) Test " + test_tree_names[0][5:-5] + " GeV (Falling) Test 10_90 GeV (Falling) Test", bins, xmin, xmax)
    deltahist.GetXaxis().SetTitle("p_{T true} - p_{T predicted}")
    deltahist.GetYaxis().SetTitle("Counts")
    deltahist.GetXaxis().SetRangeUser(-50, 50)
    deltahist.GetYaxis().SetRangeUser(0,750)
    deltahist.SetMarkerStyle(39)
    deltahist.SetMarkerColor(2)
    csvfile = output_directory + output_fil + "_train_ptbias" + str(l) + "_test_ptbias" + str(8) +"_corrected_" + str(len(indices)) + "Features_set" + str(ptrange[k][0])+ "_" + str(ptrange[k][1]) + ".csv"
    f = open(csvfile, "w")
    for j in range(len(corvals)):
        if truevals[j] < ptrange[k][0] or truevals[j] > ptrange[k][1]:
            continue
        deltahist.Fill((corvals[j][0] - truevals[j]))
        f.write(str((corvals[j][0]-truevals[j])) + "\n")
    deltahist.SetDirectory(0)
    outf.cd()
    deltahist.Write()
    outf.Write()
    outf.Close()
    f.close()
    return 0

In [6]:
def evaluate_model(model, indices, X_test, y_test, k, p, l, q):
    newX_train = []
    for i in range(len(X_train[q])):
        ph = []
        for j in range(len(X_train[q][0])):
            if j in indices:
                ph.append(X_train[q][i][j])
        newX_train.append(ph)
    model.fit(newX_train, y_train[q])
    print(model.coef_)
    print(model.intercept_)
#     elif(q%5 == 1):
#         print(model.feature_importances_)
    newX_test = []
    for i in range(len(X_test)):
        ph = []
        for j in range(len(X_test[i])):
            if j in indices:
                ph.append(X_test[i][j])
        newX_test.append(ph)
    test(newX_test, y_test, indices, model, k, p, l, q)

In [7]:
test_directory     = "./"
test_file_names    = []
group = 2
ptrange = [[10,12],[18,22],[28,32],[38,42],[48,52],[58,62],[68,72],[78,82],[88,90],[10,20],[20,30],[30,40],[40,50],[50,60],[60,70],[70,80],[80,90],[10,90]]
#for power in powers:
test_file_names.append("./prepped/ML_Prep_" + "10_90" + "_Test" + str(8) + ".csv")# teststring + "_Test" + str(8) + ".csv")
# test_file_names    = ["ML_Prep_40_60_Test.root"]
test_tree_names    = ["Tree_" + teststring + "_Test"]
# test_tree_names    = ["Tree_40_60_Test"]
test_file_paths    = [test_directory + test_file_names[i] for i in range(len(test_file_names))]
print(test_file_paths)
output_directory   = "./results/"
output_file_name   = "ML_Results_10_90.root"
output_file_path   = output_directory + output_file_name
X_test, y_test = [], []
for i in range(len(test_file_names)):
    X, y = buildfeaturescsv(test_file_names[i])
    X_test.append(X)
    y_test.append(y)

    
indices = [[0,1,2,3,4,5,6,7,8,9,10,11]]
indices.append([1,3,4])
indices.append([1])
lr = LinearRegression()
rf = RandomForestRegressor()
nn = MLPRegressor()
el = ElasticNet()
scaler = StandardScaler()

scaler = scaler.fit(X_train[0])
X_train[0] = scaler.transform(X_train[0])
X_test_scaled = scaler.transform(X_test[0])
print(X_train[0][0])
print(X_test[0][0])

['././prepped/ML_Prep_10_90_Test8.csv']
[-1.21328541 -0.29193874  0.34786865  0.14706402  1.534408   -0.48470092
  0.00688166  3.14200985 -0.47186735 -0.48191403 -0.51925244 -0.45299292]
[18.0342, 73.6365, 0.504899, 18.2868, 74.0, 0.749783, 0.650074, 110.126, 2.55784, 2.07988, 2.05664, 1.86483]


In [8]:
print(X_test[0][0])
print(y_test[0][0])
print(X_train[0][0])

[18.0342, 73.6365, 0.504899, 18.2868, 74.0, 0.749783, 0.650074, 110.126, 2.55784, 2.07988, 2.05664, 1.86483]
15.9986
[-1.21328541 -0.29193874  0.34786865  0.14706402  1.534408   -0.48470092
  0.00688166  3.14200985 -0.47186735 -0.48191403 -0.51925244 -0.45299292]


In [9]:
# nn = [keras.Sequential(),keras.Sequential(),keras.Sequential()]
# nn[0].add(layers.Dense(16, input_shape=(12,)))#, activation='tanh'))
# nn[0].add(layers.Dense(8))#, activation='tanh'))
# nn[0].add(layers.Dense(10))
# nn[0].add(layers.Dense(5))
# nn[0].add(layers.Dense(1, activation='relu'))
# nn[0].compile(optimizer='sgd', loss='mse', metrics=[r2])
# nn[1].add(layers.Dense(3, input_shape=(3,), activation='tanh'))
# nn[1].add(layers.Dense(1, activation='relu'))
# nn[1].compile(optimizer='sgd', loss='mse', metrics=[r2])
# nn[2].add(layers.Dense(1, input_shape=(1,), activation='tanh'))
# nn[2].compile(optimizer='sgd', loss='mse', metrics=[r2])
for j in range(len(X_train)):
    for i in range(len(indices)):
        for k in range(len(ptrange)):
            #evaluate_model(nn[i], indices[i], X_test[k], y_test[k], i, powers[j], k)
            evaluate_model(lr, indices[i], X_test_scaled, y_test[0],i, powers[j], k, j)
#             q += 1
#             evaluate_model(rf, indices[i], X_test[k], y_test[k],i, powers[j],k)
#             q += 1
#             evaluate_model(nn, indices[i], X_test[k], y_test[k],i, powers[j],k)
#             q += 1
            fill_schist(X_test[0], y_test[0], indices[i], i, powers[j], k)
#             q += 1
#             evaluate_model(el ,indices[i], X_test[k], y_test[k], i, powers[j],k)
#             q += 1
            print("Train set " + str(j) + ", " + str(len(indices[i])) + " features, pt range " + str(k) + " done.")

[  1.76027324  32.59590468   0.05307521 -10.70141823  -6.07802424
   0.13948467  -0.74869679  -0.29161165  -0.33541762  -0.10629284
  -0.0439453    0.7273631 ]
36.995296572591506
Train set 0, 12 features, pt range 0 done.
[  1.76027324  32.59590468   0.05307521 -10.70141823  -6.07802424
   0.13948467  -0.74869679  -0.29161165  -0.33541762  -0.10629284
  -0.0439453    0.7273631 ]
36.995296572591506
Train set 0, 12 features, pt range 1 done.
[  1.76027324  32.59590468   0.05307521 -10.70141823  -6.07802424
   0.13948467  -0.74869679  -0.29161165  -0.33541762  -0.10629284
  -0.0439453    0.7273631 ]
36.995296572591506
Train set 0, 12 features, pt range 2 done.
[  1.76027324  32.59590468   0.05307521 -10.70141823  -6.07802424
   0.13948467  -0.74869679  -0.29161165  -0.33541762  -0.10629284
  -0.0439453    0.7273631 ]
36.995296572591506
Train set 0, 12 features, pt range 3 done.
[  1.76027324  32.59590468   0.05307521 -10.70141823  -6.07802424
   0.13948467  -0.74869679  -0.29161165  -0.33